# Notes

chars contained '$' and '3'. The former is a single typo and is fixed, the latter is included in a footer for "3 KING HENRY VI", the name of a play, and has been removed everywhere.

In [6]:
from pathlib import Path
notebook_location_dir = Path.cwd()
project_root = notebook_location_dir.parent

data_path = project_root / "data"

with open(data_path / "tinyshakespeare.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [7]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1114962


In [8]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !&',-.:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
63


In [10]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

def encode(s: str) -> list[int]:
    """encoder: take a string, output a list of integers"""
    return [stoi[c] for c in s]

def decode(ls: list[int]) -> str:
    """decoder: take a list of integers, output a string"""
    return ''.join([itos[i] for i in ls]) # 

encoding = encode("Hello there!")
print(encoding)
print(decode(encoding))

[18, 41, 48, 48, 51, 1, 56, 44, 41, 54, 41, 2]
Hello there!


In [11]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

/Users/coenrouwmaat/Python/Projects/ai-safety/gpt/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([1114962]) torch.int64
tensor([16, 45, 54, 55, 56,  1, 13, 45, 56, 45, 62, 41, 50,  8,  0, 12, 41, 42,
        51, 54, 41,  1, 59, 41,  1, 52, 54, 51, 39, 41, 41, 40,  1, 37, 50, 61,
         1, 42, 57, 54, 56, 44, 41, 54,  5,  1, 44, 41, 37, 54,  1, 49, 41,  1,
        55, 52, 41, 37, 47,  7,  0,  0, 11, 48, 48,  8,  0, 29, 52, 41, 37, 47,
         5,  1, 55, 52, 41, 37, 47,  7,  0,  0, 16, 45, 54, 55, 56,  1, 13, 45,
        56, 45, 62, 41, 50,  8,  0, 35, 51, 57,  1, 37, 54, 41,  1, 37, 48, 48,
         1, 54, 41, 55, 51, 48, 58, 41, 40,  1, 54, 37, 56, 44, 41, 54,  1, 56,
        51,  1, 40, 45, 41,  1, 56, 44, 37, 50,  1, 56, 51,  1, 42, 37, 49, 45,
        55, 44, 10,  0,  0, 11, 48, 48,  8,  0, 28, 41, 55, 51, 48, 58, 41, 40,
         7,  1, 54, 41, 55, 51, 48, 58, 41, 40,  7,  0,  0, 16, 45, 54, 55, 56,
         1, 13, 45, 56, 45, 62, 41, 50,  8,  0, 16, 45, 54, 55, 56,  5,  1, 61,
        51, 57,  1, 47, 50, 51, 59,  1, 13, 37, 45, 57, 55,  1, 23, 37, 54, 39,
      

In [12]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [13]:
block_size = 8
train_data[:block_size+1]

tensor([16, 45, 54, 55, 56,  1, 13, 45, 56])

In [28]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")

when input is tensor([16]), the target is 45
when input is tensor([16, 45]), the target is 54
when input is tensor([16, 45, 54]), the target is 55
when input is tensor([16, 45, 54, 55]), the target is 56
when input is tensor([16, 45, 54, 55, 56]), the target is 1
when input is tensor([16, 45, 54, 55, 56,  1]), the target is 13
when input is tensor([16, 45, 54, 55, 56,  1, 13]), the target is 45
when input is tensor([16, 45, 54, 55, 56,  1, 13, 45]), the target is 56


In [37]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # Random starting position in the data, sampled [batch_size] times
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {decode(context.tolist())}, the target is {itos[int(target)]}")
        # print(f"when input is {context.tolist()}, the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[41, 55,  1, 44, 45, 49,  1, 42],
        [50, 51, 56,  1, 41, 50, 40, 57],
        [59, 37, 47, 41,  1, 56, 45, 48],
        [58, 41,  1, 40, 37, 54, 47,  1]])
targets:
torch.Size([4, 8])
tensor([[55,  1, 44, 45, 49,  1, 42, 37],
        [51, 56,  1, 41, 50, 40, 57, 54],
        [37, 47, 41,  1, 56, 45, 48, 48],
        [41,  1, 40, 37, 54, 47,  1, 40]])
----
when input is e, the target is s
when input is es, the target is  
when input is es , the target is h
when input is es h, the target is i
when input is es hi, the target is m
when input is es him, the target is  
when input is es him , the target is f
when input is es him f, the target is a
when input is n, the target is o
when input is no, the target is t
when input is not, the target is  
when input is not , the target is e
when input is not e, the target is n
when input is not en, the target is d
when input is not end, the target is u
when input is not endu, the target is r
when input is w, t

In [35]:
print(xb) # our input to the transformer

tensor([[41, 55,  1, 44, 45, 49,  1, 42],
        [50, 51, 56,  1, 41, 50, 40, 57],
        [59, 37, 47, 41,  1, 56, 45, 48],
        [58, 41,  1, 40, 37, 54, 47,  1]])


In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)  # TODO: research further how this works

    def forward(self, idx: Tensor, targets: Tensor | None = None) -> tuple[Tensor, Tensor]:

        # idx and targets are both (B,T) tensor of integers
        logits: Tensor = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # Batch size, Time steps, Channel (embedding size / # features)
            logits = logits.view(B*T, C)  # TODO: intuitive explanation for this reshape
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: Tensor, max_new_tokens: int) -> Tensor:
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))  # first item from batch

torch.Size([32, 63])
tensor(4.8558, grad_fn=<NllLossBackward0>)

Ue;m:TYh
LwajmRFdGSdcKjNW!aFGV!&,htCLTYFEPQ!xuWeyslIELK&PjN&
mt&ZLldYmhy;ebIqeh
 -I-gN!BCFpRwj;mUigw


In [39]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [61]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())  # plateaus around 2.5

2.501077651977539


In [62]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ANGowint te wn in,

Mait sthide ur whay! t g t'le fry,
QUFoly, rense, f le powh termathus t heby gobelllaruce che ipar:
ca mand onand th m Heerfit s be?
KEEES: lll, clou held wal ju,
Rothuly so, othe ny Hano ORinge en'sp trthe me, thisay plpsalle, INEThtyor ge s me.
ORO: serdisthatod
Whyothathase, cusee.
Tord
I pe.
Pe;
werue ize lime werey o ar Ise pad I
KELAsupant dspre funga f at ff f carind hanove h
Ag.
O be br, mal r foutt'd was, baradoume, ormesthe, shee mesth velludanen e YO:


LOREd fonos
